In [ ]:
import torch
import torch.nn.functional as F

words = open('_inputs/names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [2]:
# Building the Vocabulary of the characters
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)
print(stoi)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


## **Building a Simple Neural Net**

In [ ]:
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        xs.append(stoi[ch1]) # input is one character
        ys.append(stoi[ch2]) # output is the next character

In [50]:
xs_enc = torch.nn.functional.one_hot(torch.tensor(xs), num_classes=len(stoi)).float() # N x 27
ys_enc = torch.nn.functional.one_hot(torch.tensor(ys), num_classes=len(stoi)).float() # N x 27

W = torch.randn(len(stoi), len(stoi), requires_grad=True) # 27 x 27

In [52]:
for k in range(100):
    logits = xs_enc @ W # N x 27
    counts = logits.exp()
    probs = counts / counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(len(ys)), ys].log().mean()

    if k % 10 == 0:
        print(loss.item())
    
    W.grad = None
    loss.backward()
    
    W.data -= 10 * W.grad

3.7425448894500732
3.183295726776123
2.9403176307678223
2.807241916656494
2.727997303009033
2.6769602298736572
2.6415903568267822
2.615656852722168
2.595832109451294
2.5801756381988525


In [82]:
# going in inference to generate names
for _ in range(10):
    name = '.'
    while True:
        ch = name[-1]
        x = torch.nn.functional.one_hot(torch.tensor([stoi[ch]]), num_classes=len(stoi)).float()
        logits = x @ W
        probs = logits.exp()
        next_ch = torch.multinomial(probs, 1).item()
        name += itos[next_ch]
        if name[-1] == '.':
            break
    print(name)

.jnattis.
.iai.
.niyfa.
..
.n.
.aridan.
.arel.
.lamadai.
.ayawenaiucyauseneeli.
.mza.
